In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file="notMNIST.pickle"
with open(pickle_file,'rb') as f:
    full_data=pickle.load(f)
    train_dataset=full_data['train_dataset']
    train_labels=full_data['train_labels']
    test_dataset=full_data['test_dataset']
    test_labels=full_data['test_labels']
    valid_dataset=full_data['valid_dataset']
    valid_labels=full_data['valid_labels']
    del full_data
    print("train set:",train_dataset.shape,train_labels.shape)
    print("test set:",test_dataset.shape,test_labels.shape)
    print("valid set:",valid_dataset.shape,valid_labels.shape)

train set: (200000, 28, 28) (200000,)
test set: (10000, 28, 28) (10000,)
valid set: (10000, 28, 28) (10000,)


In [3]:
# Reshape the dataset
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions,labels):
    return(100.0*np.sum(np.argmax(predictions,1)==np.argmax(labels,1))/predictions.shape[0])

In [6]:
#initialize
batch_size=16
patch_size=5
depth=16
more_depth=32
num_hidden=64
image_size=28


graph=tf.Graph()
with graph.as_default():
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    # Weight and Bias Parameters
    K=8
    ###### Layer 1
    w1=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,num_channels,K],stddev=0.1)) ##### W[5,5,1,4] ->[,28,28,4]
    b1=tf.Variable(tf.zeros([K]))
    
    ###### Layer 2
    w2=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,K,depth],stddev=0.1))  ##### W[5,5,4,16] ->[,14,14,16]
    b2=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    ###### Layer 3
    w3=tf.Variable(tf.truncated_normal(shape=[patch_size,patch_size,depth,more_depth],stddev=0.1))  ###### W[5,5,16,16] ->[,7,7,16]
    b3=tf.Variable(tf.constant(1.0,shape=[more_depth]))
    
    ###### connected Layer
    w4=tf.Variable(tf.truncated_normal(shape=[image_size//4 * image_size//4 * more_depth,num_hidden],stddev=0.1)) ##### W[7*7*16,64] ->[,64]
    b4=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    ###### final layer
    w5=tf.Variable(tf.truncated_normal(shape=[num_hidden,num_labels],stddev=0.1)) ####### W[64,10]
    b5=tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    # Computing CNN model
    def model(data):
        conv1=tf.nn.relu(tf.nn.conv2d(data,w1, strides=[1,1,1,1],padding='SAME') +b1)
        
        conv2=tf.nn.relu(tf.nn.conv2d(conv1,w2, strides=[1,2,2,1], padding='SAME') +b2)
        
        conv3=tf.nn.relu(tf.nn.conv2d(conv2,w3, strides=[1,2,2,1], padding='SAME') +b3)
        
        conv3_shape=conv3.get_shape().as_list()
        conv3_reshape=tf.reshape(conv3, [conv3_shape[0],conv3_shape[1]*conv3_shape[2]*conv3_shape[3]])
        
        #connected Layer
        connec=tf.nn.relu(tf.matmul(conv3_reshape,w4) +b4)
        
        return (tf.matmul(connec,w5) +b5)
    
    # Train Computation
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    # Optimizer
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction=tf.nn.softmax(logits)
    test_prediction=tf.nn.softmax(model(tf_test_dataset))
    valid_prediction=tf.nn.softmax(tf_valid_dataset)
        
    


In [7]:
# Iterate the model and train 
num_steps=100

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialised......")
    for step in range(num_steps):
        offset=np.random.randint(0,200000-16)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        opt,l,pred=sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step%10==0):
            print("loss at %d:%f" %(step,l))
            print("Mini Batch Accuracy:%.2f%%" %accuracy(pred,batch_labels))
            print("Validation Accuracy:%.2f%%" %accuracy(valid_prediction.eval(),valid_labels))
    print("-"*30)
    
    print("Test Accuracy:%.2f%%" %accuracy(test_prediction.eval(),test_labels))
            

Initialised......
loss at 0:4.271260
Mini Batch Accuracy:25.00%
Validation Accuracy:2800000.00%
loss at 10:3.846241
Mini Batch Accuracy:12.50%
Validation Accuracy:2800000.00%
loss at 20:2.324955
Mini Batch Accuracy:12.50%
Validation Accuracy:2800000.00%
loss at 30:2.346263
Mini Batch Accuracy:12.50%
Validation Accuracy:2800000.00%
loss at 40:2.417567
Mini Batch Accuracy:6.25%
Validation Accuracy:2800000.00%
loss at 50:2.212980
Mini Batch Accuracy:25.00%
Validation Accuracy:2800000.00%
loss at 60:2.376173
Mini Batch Accuracy:6.25%
Validation Accuracy:2800000.00%
loss at 70:2.270123
Mini Batch Accuracy:25.00%
Validation Accuracy:2800000.00%
loss at 80:2.305731
Mini Batch Accuracy:12.50%
Validation Accuracy:2800000.00%
loss at 90:2.365678
Mini Batch Accuracy:12.50%
Validation Accuracy:2800000.00%
------------------------------
Test Accuracy:10.00%
